In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from time import sleep

In [2]:
# собираем названия всех регионалльных центров
regional_centers = []
my_response = requests.get('https://marketing-course.ru/administrativnye-centry-regionov/').text
soup = BeautifulSoup(my_response, 'lxml')
table = soup.find_all('table')[0]
list_regions_info = table.find_all('tr')[1:]
for region in list_regions_info:
    title_regional_center = region.find_all('td')[3].text
    regional_centers.append(title_regional_center)
print(regional_centers)
len(regional_centers)

['Белгород', 'Брянск', 'Владимир', 'Воронеж', 'Иваново', 'Калуга', 'Кострома', 'Курск', 'Липецк', 'Москва', 'Москва', 'Орел', 'Рязань', 'Смоленск', 'Тамбов', 'Тверь', 'Тула', 'Ярославль', 'Архангельск', 'Вологда', 'Калининград', 'Петрозаводск', 'Сыктывкар', 'Санкт-Петербург', 'Мурманск', 'Салехард', 'Великий Новгород', 'Псков', 'Санкт-Петербург', 'Уфа', 'Киров', 'Йошкар-Ола', 'Саранск', 'Нижний Новгород', 'Оренбург', 'Пенза', 'Пермь', 'Самара', 'Саратов', 'Казань', 'Ижевск', 'Ульяновск', 'Чебоксары', 'Курган', 'Екатеринбург', 'Тюмень', 'Ханты-мансийск', 'Челябиннск', 'Салехард', 'Горно-Алтайск', 'Барнаул', 'Улан-Удэ', 'Чита', 'Иркутск', 'Кемерово', 'Красноярск', 'Новосибирск', 'Омск', 'Томск', 'Кызыл', 'Абакан', 'Благовещенск', 'Биробиджан', 'Петропавловск-Камчатский', 'Магадан', 'Владивосток', 'Якутск', 'Южно-Сахалинск', 'Хабаровск', 'Анадырь', 'Майкоп', 'Астрахань', 'Волгоград', 'Элиста', 'Краснодар', 'Ростов', 'Махачкала', 'Магас', 'Нальчик', 'Черкесск', 'Владивкавказ', 'Ставрополь'

83

In [3]:
titles = []
adresses = []
websites = []
phone_numbers = []
emails = []

In [4]:
def input_info(): # поиск информации
    input_tab = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/input")
    input_tab.send_keys(regional_center)
    
def click_search_button(): # нажатие на кнопку поиска
    search_button = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[2]")
    search_button.click()

def click_clear_info_button(): # нажатие на кнопку удаления текста в области поиска
    clear_info_button = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[1]')
    clear_info_button.click()
    
    
def collect_data(): # сбор всех данных(адрес, сайт, телефон, почта) с найденного представителя поддержки бизнеса 
    try:
        title = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/div').text
        adress = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[1]/td[2]').text
        website = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[2]/td[2]/a').get_attribute('href')
        phone = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[3]/td[2]').text
        email = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[4]/td[2]/a').text
    except NoSuchElementException:
        title = None
        adress = None
        website = None
        phone = None
        email = None 
    titles.append(title)
    adresses.append(adress)
    websites.append(website)
    phone_numbers.append(phone)
    emails.append(email)

In [5]:
browser = Chrome()
url = 'https://xn--l1agf.xn--p1ai/services/reg-support-map/'
browser.maximize_window() 
browser.get(url)
sleep(2) # пауза, чтобы страницы успела прогрузиться 
for regional_center in regional_centers:
    input_info()
    click_search_button()
    sleep(1)
    collect_data()
    click_clear_info_button()

In [6]:
data = {'название': titles, 'адрес': adresses, 'сайт': websites, 'телефон': phone_numbers, 'почта': emails}
df = pd.DataFrame(data=data)

In [7]:
df

,название,адрес,сайт,телефон,почта
0,ФОНД МКК БОФПМСП,"Белгородская область, Белгород, Королева, д. 2...",https://www.mb31.ru/,+74722 52-71-43,fond-biznesa@yandex.ru
1,ГАУ БО ЦОУ «Мой бизнес»,"Брянская, Брянск, Бежицкая, 54, 241023",https://xn--32-9kcqjffxnf3b.xn--p1ai/,+7 (4832) 58-92-77,info@mybiz32.ru
2,Государственное автономное учреждение Владимир...,"Владимирская, Владимир, Октябрьский, 21, 600005",https://xn---33-9cdulgg0aog6b.xn--p1ai/,84922777620,info@mb33.ru
3,АНО «ЦПП ВО»,"Воронежская, Воронеж, Свободы, 21, 1/9, 394018",http://moibiz36.ru/,+7 473 20-70-100,info@moibiz36.ru
4,АНО Центр развития предпринимательства и подде...,"Ивановская, Иваново, Шереметевский, 85Г, 153037",http://xn--37-9kcqjffxnf3b.xn--p1ai/,+7 4932 66-67-67,info@moybiznes37.ru
...,...,...,...,...,...
78,Региональный фонд «ЦПП КБР»,"Кабардино-Балкарская Республика, Нальчик, Киро...",https://cppkbr.ru/,+7 (8662) 72-21-81,fppkbr@inbox.ru
79,АНО «Центр «Мой бизнес» КЧР»,"Карачаево-Черкесская, Черкесск, Ленина, 53, 36...",http://businesskchr.ru/,88782250227,cpp-kchr@mail.ru
80,None,None,None,None,None
81,НО ФППСК,"Ставропольский край, Ставрополь, Голенева, 73А...",http://fppsk26.ru/,+7 865 223-56-20,info@fppsk26.ru
